In [2]:
import os
import numpy as np
from numpy.random import rand 
import torch

In [3]:
np_array = rand(4,3)
print(type(np_array),np_array.shape)
tensor = torch.FloatTensor(np_array)
print(type(tensor),tensor.shape)

<class 'numpy.ndarray'> (4, 3)
<class 'torch.Tensor'> torch.Size([4, 3])


In [4]:
############################################ Process on Tensor #######################################################

In [5]:
#torch.cat() 
#torch.Tensor.expand()
#torch.Tensor.squeeze() 
#torch.Tensor.repeat()
#torch.Tensor.narrow() 
#torch.Tensor.view()
#torch.Tensor.resize_() 
#torch.Tensor.permute()

In [18]:
# torch.cat((A,B),dim)<-> torch.chunck(A, dim)
# Concatenates the given sequence of seq tensors in the given dimension. 
# All tensors must either have the same shape (except in the cat dimension) or be empty.

zeros = torch.zeros(2,3)
ones = torch.ones(2,3)

cat1 = torch.cat((zeros,ones),dim = 0)
cat2 = torch.cat((zeros,ones),dim = 1)

print(cat1.size())
print(cat2.size())

torch.Size([4, 3])
torch.Size([2, 6])


In [19]:
# torch.Tensor.expand(tensor, scale)
# Returns a new view of the self tensor with singleton dimensions expanded to a larger size.
# Passing -1 as the size for a dimension means not changing the size of that dimension.

singleton = torch.randn(1,3)

expand = torch.Tensor.expand(singleton,(4,-1))

print(singleton.size())
print(expand.size())

torch.Size([1, 3])
torch.Size([4, 3])


In [17]:
# torch.Tensor.squeeze() = torch.squeeze(tensor, dim) <--> torch.unsqueeze(dim)
# Returns a tensor with all the dimensions of input of size 1 removed.

sigleton1 = torch.randn(3,4,1,1)

squeeze1 = torch.squeeze(sigleton1)
squeeze2 = torch.Tensor.squeeze(sigleton1)

print(squeeze1.size())
print(squeeze2.size())


torch.Size([3, 4])
torch.Size([3, 4])


In [22]:
# torch.Tensor.repeat(tesor, scale)
# Repeats this tensor along the specified dimensions.
# Unlike expand(), this function copies the tensor’s data.

zeros = torch.zeros(3,2)

repeat1 = torch.Tensor.repeat(zeros,(4,2,2))#= repeat2 = zeros.repeat(4,2,2,1)

print(zeros.size())
print(repeat1.size())

torch.Size([3, 2])
torch.Size([4, 6, 4])


In [21]:
# torch.Tensor.narrow(dimension, start, length) 
# Returns a new tensor that is a narrowed version of self tensor. 
# The dimension dim is narrowed from start to start + length. 
# The returned tensor and self tensor share the same underlying storage.
# You can also think we pick a specific slices from the original input

input_tensor = torch.Tensor(([1,2,3],[4,5,6]))

narrow = torch.Tensor.narrow(input_tensor,1,1,2)

print(input_tensor)
print(narrow)

tensor([[1., 2., 3.],
        [4., 5., 6.]])
tensor([[2., 3.],
        [5., 6.]])


In [69]:
# torch.Tensor.view(tensor, scale)
# Returns a new tensor with the same data as the self tensor but of a different size.
# The tensor must be contiguous() to be viewed.

input_tensor = torch.randn((3,4,5))

view1 = torch.Tensor.view(input_tensor,(12,5)) # =view2 = input_tensor.view(12,5)

print(view1.size())

torch.Size([12, 5])


In [83]:
# torch.Tensor.resize_(tensor,scale)
# Resizes self tensor to the specified size. 
# If the number of elements is larger than the current storage size, 
# then the underlying storage is resized to fit the new number of elements. 
# If the number of elements is smaller, the underlying storage is not changed. 
# Existing elements are preserved but any new memory is uninitialized.

input_tensor1 = torch.randn(3,2)

view = input_tensor1.view(2,3)
print('input', input_tensor1.size())
print('view ',view.size())

resize2 = torch.Tensor.resize_(input_tensor1,(2,2))
resize1 = torch.Tensor.resize_(input_tensor1,(4,4))


print('input',input_tensor1.size()) ###find the difference between view() and resize_()
print('resize1',resize1.size())
print('resize2',resize2.size())

input torch.Size([3, 2])
view  torch.Size([2, 3])
input torch.Size([4, 4])
resize1 torch.Size([4, 4])
resize2 torch.Size([4, 4])


In [85]:
# torch.Tensor.permute(tensor, scale)
# Permute the dimensions of this tensor.

input_tensor2 = torch.rand(3,4,5)
tranpose = t

permute = torch.Tensor.permute(input_tensor2,(2,0,1))

print(permute.size())

torch.Size([5, 3, 4])


In [87]:
############################################### Process on Varible ##################################################

In [24]:
from torch.autograd import Variable

In [25]:
x = Variable(torch.Tensor([10]),requires_grad=True)
y = Variable(torch.Tensor([5]),requires_grad=True)
z=x*y*5
z.backward()
print(x.grad)
print(y.grad)

tensor([25.])
tensor([50.])


In [120]:
N, D_in, H, D_out = 5, 100, 15, 10
x = Variable(torch.rand(N, D_in), requires_grad = False)
y = torch.randn((N, D_out), requires_grad = False)        # Varibles and tensor share the same API
w1 = Variable(torch.randn(D_in, H), requires_grad = True)
w2 = Variable(torch.randn(H, D_out), requires_grad = True)

In [121]:
z = 2*x
w3 = 2*w1
print(z.grad_fn)
print(w3.grad_fn)

None


In [123]:
learning_rate = 1e-4
for t in range(100):
    y_pred = x.mm(w1).clamp(min=0).mm(w2) # torch.mm() performs a matrix multiplication of the matrices mat1 and mat2.
                                          # torch.clamp clamp all elements in input into the range [min, max] and return a resulting tensor:
    loss = (y_pred - y).pow(2).sum()
    if w1.grad.sum(): w1.grad.data.zero_()
    if w1.grad.sum(): w2.grad.data.zero_()
    loss.backward()
    
    w1.data -= learning_rate * w1.grad.data #.data pick the tensor value, .item() pick the 1-D tensor's value as an numpy number
    w2.data -= learning_rate * w2.grad.data

In [125]:
################################# Process on Module ##########################################

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [27]:
class Net(nn.Module):
    def __init__(self):
        # 1 input image channel, 6 output channels, 5x5 square convolution
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,6,5) # input of nn.Conv2d is a 4D tensor of nSamples x nChannels x Height x Width.
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = x.view(-1, self.num_flat_features(x)) # x.view(x.numel()) has the same function
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)                 
        return x
    
    def num_flat_features(self,x):
        size = x.size()[1:] # do not want to count batch_size
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)
params = list(net.parameters())

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [29]:
# input defination
input = torch.randn(1,1,32,32)
output = net(input)

In [30]:
# label and loss defination
target = torch.randn(10)
print(input)
print(target)
target = target.view(1,-1)
criterion = nn.MSELoss()
loss = criterion(output, target)

tensor([[[[-0.7052, -0.5608,  0.9386,  ...,  0.3902,  1.1509,  1.9225],
          [-0.5723,  0.4903, -0.6007,  ...,  0.0484, -0.8165,  1.5653],
          [ 0.6350,  0.7251,  0.5297,  ...,  0.7496, -0.0448, -1.5141],
          ...,
          [ 0.4189, -0.1265, -2.4058,  ..., -0.3103, -0.1630,  1.0984],
          [ 0.6356,  0.2582, -0.8845,  ..., -0.1507, -0.8808,  0.7621],
          [-0.1734, -0.0352,  0.7242,  ...,  0.2539, -0.8891,  1.3322]]]])
tensor([ 1.4428,  0.4261,  1.0781, -0.7588, -0.7374, -0.1408, -1.5827, -1.0786,
         0.1833, -1.2377])


In [31]:
# backprop and weights updates
net.zero_grad()
loss.backward()
print(loss.grad_fn)

learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [39]:
# backprop with torch.optim

import torch.optim as optim

# create SGD optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update


print(net.conv1.bias.grad)

torch.Size([1, 16, 5, 5])
torch.Size([16, 5, 5])
tensor([ 0.0067, -0.0125,  0.0078,  0.0244,  0.0202,  0.0027])
